In [ ]:
import polars as pl
import validate.lesion as vl
import validate.general as vg

from constants import list_id_cols

import utils
from pathlib import Path
import os

PATH_INPUT = os.getenv("PATH_INPUT")

lf = utils.get_df(Path(PATH_INPUT).joinpath("test data.accdb")).lazy()

lf2 = utils.get_df(Path(PATH_INPUT).joinpath("ZON CHERAS OC DATABASE JULY-OCT 2023 edit MAIwani.accdb")).lazy()

obj = vg.ValidationGeneral(lf, "test data")
lfall = pl.concat([lf, lf2])

In [ ]:
lfall.columns

0 - No such habit
1- habit currently practiced
2 - past habit now has stopped (minimum 6 months)

In [ ]:
str_habit_false = "0 - No such habit"

lfall.select("HABITS", "TOBACCO", "BBETEL QUID CHEWING", "ALCOHOL").with_columns(
  pl.when(
    (pl.col("HABITS") == True)
    & (pl.any_horizontal(pl.col("TOBACCO", "BBETEL QUID CHEWING", "ALCOHOL").is_null()))
  )
  .then(False)
  .otherwise(True)
  .alias("valid_completeness")
).with_columns(
  pl.when(pl.col("TOBACCO").is_null() | pl.col("TOBACCO").eq(str_habit_false))
  .then(False)
  .otherwise(True)
  .alias("has_tobacco"),
  pl.when(
    pl.col("BBETEL QUID CHEWING").is_null()
    | pl.col("BBETEL QUID CHEWING").eq(str_habit_false)
  )
  .then(False)
  .otherwise(True)
  .alias("has_betel"),
  pl.when(pl.col("ALCOHOL").is_null() | pl.col("ALCOHOL").eq(str_habit_false))
  .then(False)
  .otherwise(True)
  .alias("has_alcohol"),
).with_columns(
  pl.when(
    pl.col("HABITS") == pl.any_horizontal(["has_tobacco", "has_betel", "has_alcohol"])
  )
  .then(True)
  .otherwise(False)
  .alias("valid_habits")
).filter(pl.col("valid_habits") == False).collect()


In [ ]:
habit_whitelist = [
    "0 - No such habit|Null|Null",
    "1- habit currently practiced|Yes|Yes",
    "1- habit currently practiced|Yes|No",
    "2 - past habit now has stopped (minimum 6 months)|Yes|Null",
]

In [ ]:
lfall.select(
  pl.col("ALCOHOL", "ALCOHOL ADVISED", "ALCOHOL QUIT").fill_null("Null")
).with_columns(
  pl.concat_str(
    pl.col("ALCOHOL", "ALCOHOL ADVISED", "ALCOHOL QUIT"), separator="|"
  ).alias("habit_concat")
).filter(~pl.col("habit_concat").is_in(habit_whitelist)).collect()
